In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation  import train_test_split

In [2]:
train = pd.read_csv("train.csv",sep = ',')
store = pd.read_csv("store.csv",sep = ',')
test = pd.read_csv("test.csv",sep = ',')

/Library/Python/2.7/site-packages/pandas-0.16.0-py2.7-macosx-10.10-intel.egg/pandas/io/parsers.py:1164: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
def toBinary(featureCol, df):
    values = set(df[featureCol].unique())
    newCol = [featureCol + val for val in values]
    for val in values:
        df[featureCol + val] = df[featureCol].map(lambda x: 1 if x == val else 0)
    return newCol


def build_features(features,data):
    data.fillna(0,inplace = True)
    data.loc[data.Open.isnull(),'Open']=1
    features.extend(['Store','CompetitionDistance','CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Promo','Promo2','Promo2SinceYear'])
    features.append('SchoolHoliday')
    data['SchoolHoliday'] = data['SchoolHoliday'].astype(float)
    features.append('StateHoliday')
    data.loc[data['StateHoliday'] == 'a','StateHoliday']='1'
    data.loc[data['StateHoliday'] == 'b','StateHoliday']='2'
    data.loc[data['StateHoliday'] == 'c','StateHoliday']='3'
    data['StateHoliday'] = data['StateHoliday'].astype(float)
    train['Date'] = pd.to_datetime(train['Date'])
    test['Date'] = pd.to_datetime(test['Date'])
    features.append('DayOfWeek')
    features.append('Month')
    features.append('Day')
    features.append('Year')
    data['Year'] = data.Date.dt.year
    data['Month'] = data.Date.dt.month
    data['Day'] = data.Date.dt.day
    data['DayOfWeek'] = data.Date.dt.dayofweek
    for x in ['a','b','c','d']:
        features.append('StoreType' + x)
        data['StoreType' + x] = data['StoreType'].map(lambda y: 1 if y==x else 0)
    newCol = toBinary('Assortment',data)
    features +=newCol
    

In [4]:
train.fillna(1, inplace = True)
test.fillna(1, inplace = True)

In [5]:
train = train[train["Open"]!=0]
train = train[train["Sales"]>0]

In [6]:
print len(train)
print list(train.columns)

844338
['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']


In [7]:
train = pd.merge(train,store, on='Store')
test = pd.merge(test,store,on = 'Store')

In [8]:
print train.columns
print train.head()

Index([u'Store', u'DayOfWeek', u'Date', u'Sales', u'Customers', u'Open', u'Promo', u'StateHoliday', u'SchoolHoliday', u'StoreType', u'Assortment', u'CompetitionDistance', u'CompetitionOpenSinceMonth', u'CompetitionOpenSinceYear', u'Promo2', u'Promo2SinceWeek', u'Promo2SinceYear', u'PromoInterval'], dtype='object')
   Store  DayOfWeek        Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5  2015-07-31   5263        555     1      1            0   
1      1          4  2015-07-30   5020        546     1      1            0   
2      1          3  2015-07-29   4782        523     1      1            0   
3      1          2  2015-07-28   5011        560     1      1            0   
4      1          1  2015-07-27   6102        612     1      1            0   

   SchoolHoliday StoreType Assortment  CompetitionDistance  \
0              1         c          a                 1270   
1              1         c          a                 1270   
2              1       

In [9]:
features = []
build_features(features,train)
build_features([],test)

In [10]:
X_train, X_valid = train_test_split(train, test_size = 0.012)
y_train = X_train.Sales
y_valid = X_valid.Sales
 

In [11]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [ ]:
model.fit(X_train[features],y_train)

In [ ]:
print train.head()

In [120]:
#print X_train["DayOfWeek"]

200321    0
819018    5
51212     3
464021    5
590091    3
814207    2
148514    5
182389    3
91153     5
651035    4
769023    1
669944    4
421978    2
428374    5
78150     2
787624    1
629808    3
659992    4
63522     4
21051     1
333647    5
361946    2
667343    1
777490    5
623319    0
696277    3
521260    3
250782    2
619729    4
383047    2
         ..
19576     5
284295    1
441243    1
363781    1
656307    5
465185    5
219043    2
31564     5
264570    2
362889    0
518575    2
149366    5
342858    4
648284    2
676777    5
210053    3
554370    2
425304    2
452717    4
35065     2
459717    2
382158    3
452003    5
169559    4
115242    3
37603     2
74741     1
165080    4
605922    5
157173    2
Name: DayOfWeek, dtype: int64


In [98]:
print X_train

        Store  DayOfWeek       Date  Sales  Customers  Open  Promo  \
414977    549          1 2013-04-09   5046        592     1      1   
546662    725          3 2015-02-12   5602        566     1      0   
680519    901          3 2013-01-17   5616        579     1      0   
419495    555          1 2013-04-23   5526        504     1      1   
429577    568          3 2013-12-05   6936        512     1      1   
800456   1060          2 2014-12-17  10638        982     1      1   
714567    947          4 2014-08-22   7304       1038     1      1   
450917    596          3 2013-11-21   4280        619     1      1   
546635    725          0 2015-03-16   6946        671     1      1   
96355     128          3 2014-04-10   6859        549     1      0   
116929    156          5 2013-10-05   2797        311     1      0   
77489     103          4 2014-06-13   4190        421     1      0   
352237    466          0 2013-10-28   4763        536     1      0   
771064   1021       

In [105]:
print store.columns

Index([u'Store', u'StoreType', u'Assortment', u'CompetitionDistance', u'CompetitionOpenSinceMonth', u'CompetitionOpenSinceYear', u'Promo2', u'Promo2SinceWeek', u'Promo2SinceYear', u'PromoInterval'], dtype='object')
